# Account iteration study (supended account)

## Import

In [ ]:
import pandas as pd

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


import sys
import importlib

sys.path.insert(0, '../')
import general_utils as gen_ut

sys.path.insert(0, '../3_Hashtag_study')
import hashtag_util as ut_ht

## Function definition

In [ ]:
dfUsers = pd.read_csv('../tweets.csv',usecols=['user_screen_name','user_code_error'])
dfUsers = dfUsers.groupby('user_screen_name').first()
dfUsers

In [ ]:
def df_preparation(cols,filename='../tweets.csv'):
    columns = list(set(cols + ['id','user_screen_name','rt_user_screen_name','in_reply_to_screen_name']))
    
    df = pd.read_csv(filename,low_memory=False, usecols=columns)
    dfSuspect = pd.DataFrame()
    for i,r in dfUsers.iterrows():
        if r['user_code_error'] == 63:
            df1 = df.loc[df['user_screen_name'] == i,:]
            dfSuspect = pd.concat([df1,dfSuspect],ignore_index=True)

            df1 = df.loc[df['rt_user_screen_name'] == i,:]
            dfSuspect = pd.concat([df1,dfSuspect],ignore_index=True)

            '''df1 = df.loc[df['in_reply_to_screen_name'] == i,:]
            dfSuspect = pd.concat([df1,dfSuspect],ignore_index=True)'''
            del df1
        
    dfSuspect.drop_duplicates(subset=['id'])
    return dfSuspect

## Link study

In [ ]:
df = df_preparation(['urls','created_at'])
df['created_at'] = pd.to_datetime(df['created_at'], format="%a %b %d %X %z %Y")
df

In [ ]:
#Creating a map of all urls with the number of uses
listUrls = []
for s in df['urls']:
    urls = gen_ut.get_string_json(s,'display_url')
    for url in urls:
        if url:
            url = url.split("//")
            url = url[0].split("/")
            listUrls.append(url[0])


dfUrls = pd.DataFrame()
dfUrls['url'] = listUrls
dfUrls['count'] = 1


dfUrls = dfUrls.groupby('url').sum()
dfUrls.sort_values(['count'], axis = 0,inplace=True,ascending=False)

dfUrls

In [ ]:
n = 20
fig = px.histogram(dfUrls.head(n),y=dfUrls.head(n).index,x='count',
            title="The most %d url used in the tweets"% n,orientation = 'h')
fig.update_yaxes(title='URL name')
fig.show()

In [ ]:
#Creation of a dictionary of num of use per date
my_dict = {"url":[],"date":[], "count":[]};
for i in range(len(df)):
    s = df.loc[i,'urls']
    d = df.loc[i,'created_at']
    url = gen_ut.get_string_json(s,'display_url')
    if url:
        url = url[0].split("//")
        url = url[0].split("/")
        
        my_dict["url"].append(url[0])
        my_dict["date"].append(d)
        my_dict["count"].append(1)

dfUseUrl = pd.DataFrame.from_dict(my_dict)
dfUseUrl['Week/Year'] = dfUseUrl['date'].apply(lambda x: "%d-%d" % (x.isocalendar()[1] , x.isocalendar()[0]))
dfUseUrl.drop(['date'], axis=1,inplace=True)

dfUseUrl = dfUseUrl.groupby(['Week/Year', 'url']).sum()
dfUseUrl.reset_index(inplace=True)

dfUseUrl['Week/Year'] =pd.to_datetime(dfUseUrl['Week/Year']+ '-1', format="%W-%Y-%w")
dfUseUrl.sort_values(['Week/Year'],axis = 0,inplace=True,ascending=True)

dfUseUrl

In [ ]:
#All in the same graphic
fig = go.Figure()

for w in dfUrls.head().index:
    mask = dfUseUrl['url'] == w
    fig.add_trace(go.Scatter(x=dfUseUrl.loc[mask,'Week/Year'], y=dfUseUrl.loc[mask,'count'],
                    mode='lines+markers',
                    name=w))
fig.update_layout(title='All url history use',xaxis_title='Date',yaxis_title='use count')

fig.show()

In [ ]:
# All in different graphic
for w in dfUrls.head().index:
    fig = go.Figure()
    mask = dfUseUrl['url'] == w
    fig.add_trace(go.Scatter(x=dfUseUrl.loc[mask,'Week/Year'], y=dfUseUrl.loc[mask,'count'],
                    mode='lines+markers',
                    name=w))
    fig.update_layout(title="History use of url '%s'"%w,xaxis_title='Date',yaxis_title='use count')

    fig.show()

## Hahstag study

In [ ]:
df = df_preparation(['hashtags','created_at'])
df['created_at'] = pd.to_datetime(df['created_at'], format="%a %b %d %X %z %Y")

In [ ]:
#Creating a map of all hashtags with the number of uses
listHashtags = []

for s in df['hashtags']:    
    [ listHashtags.append(x) for x in gen_ut.get_string_json(s,'text') ]
    
dfHashtags = pd.DataFrame()
dfHashtags['hashtags'] = listHashtags
dfHashtags['count'] = 0

dfHashtags = dfHashtags.groupby('hashtags').count()
dfHashtags.sort_values(['count'],axis = 0,inplace=True,ascending=False)

dfHashtags

In [ ]:
importlib.reload(ut_ht)
dfUse = ut_ht.process_dfUse(df)
dfUse

In [ ]:
ut_ht.visual_histogram(dfHashtags,30)

In [ ]:
sum(dfHashtags['count'])

In [ ]:
hastagRemove = ['vaccin.*','covid.*','corona.*','astrazeneca','pfizer','sarscov2','sputnikv','moderna']
dfHashtagFiltered = dfHashtags
for r in hastagRemove:
    mask = dfHashtagFiltered.index.str.lower().str.match(r) == True
    dfHashtagFiltered.drop(dfHashtagFiltered[mask].index, inplace=True)
    
dfMoreFiltered = dfHashtagFiltered
hastagRemove = ['.*lombardia.*','draghi','conte','m5s','mattarella','salvini','speranza','renzi','lega','.*governo.*',
           '.*moratti.*','zingaretti','scanzi','burioni','crisanti']
for r in hastagRemove:
    mask = dfMoreFiltered.index.str.lower().str.match(r) == True
    dfMoreFiltered.drop(dfMoreFiltered[mask].index, inplace=True)

ut_ht.visual_histogram(dfMoreFiltered,20)

In [ ]:
ut_ht.visual_by_date_together(dfMoreFiltered,dfUse)

In [ ]:
ut_ht.visual_by_date_split(dfMoreFiltered,dfUse)

## Self retweet

In [ ]:
df = df_preparation(['is_self_rt'])

In [ ]:
dfSelf = df.groupby('user_screen_name').sum()
dfSelf['all_rt'] = df.groupby('user_screen_name').count().iloc[:,0]
dfSelf.sort_values(['all_rt'],inplace=True)
dfSelf

In [ ]:
n = 20
fig = make_subplots(rows=1, cols=1)

fig.add_trace(go.Bar(y=dfSelf.tail(n).index, x=dfSelf.tail(n)['all_rt'],orientation='h',  name = 'All retweet'), row=1, col=1)
fig.add_trace(go.Bar(y=dfSelf.tail(n).index, x=dfSelf.tail(n)['is_self_rt'],orientation='h', name = 'Self retweet'), row=1, col=1)

fig.update_layout(title="How many retweet are self retweet (the most 20 retweeter)")
fig.update_xaxes(title="Count of retweets")
fig.update_yaxes(title="Username")

fig.update_layout(
    barmode="overlay",
    bargap=0.1)


fig.show()

## Iteration with other users

In [ ]:
df = df_preparation(['is_self_rt'])
df

### Retweet

In [ ]:
dfRetweet = df.dropna(subset=['rt_user_screen_name']).copy()
dfRetweet.drop(columns=['in_reply_to_screen_name'],inplace=True,errors='ignore')
dfRetweet = dfRetweet.groupby('rt_user_screen_name').count()

dfRetweet.rename(columns={'user_screen_name':'all_rt'},inplace=True,errors='ignore')
dfRetweet['self_rt'] = df.dropna(subset=['rt_user_screen_name'
                                        ]).copy().groupby('rt_user_screen_name').sum().loc[:,'is_self_rt']
dfRetweet.drop(columns=['is_self_rt'],inplace=True,errors='ignore')

dfRetweet['real_rt'] = dfRetweet['all_rt'] - dfRetweet['self_rt']
dfRetweet.sort_values('real_rt',ascending=False,inplace=True)
dfRetweet


In [ ]:
n = 20
fig =px.histogram(dfRetweet.head(n),y=dfRetweet.head(n).index,x='real_rt',orientation='h')
fig.update_yaxes(title='username')
fig.update_layout(title="The most %d users retweeted (without self retweet)"%n)
fig.show()

### Reply

In [ ]:
dfReply = df.dropna(subset=['in_reply_to_screen_name']).copy()
dfReply.drop(columns=['rt_user_screen_name'],inplace=True,errors='ignore')
dfReply.drop(columns=['is_self_rt'],inplace=True,errors='ignore')

dfReply = dfReply.groupby('in_reply_to_screen_name').count()

dfReply.rename(columns={'user_screen_name':'count'},inplace=True,errors='ignore')
dfReply.sort_values('count',ascending=False,inplace=True)

dfReply

In [ ]:
n = 20
fig =px.histogram(dfReply.head(n),y=dfReply.head(n).index,x='count',orientation='h')
fig.update_yaxes(title='username')
fig.update_layout(title="The most %d users replied"%n)
fig.show()

## Suspect iteration study

In [ ]:
df = df_preparation(['is_self_rt'])
df1 = df.copy()

df1['sum_total_posts'] = 1
df1 = df1.groupby('user_screen_name').sum()

df = df.groupby('user_screen_name').count()
df['sum_total_posts'] = df1['sum_total_posts']
df['sum_self_rt'] = df1['is_self_rt']
del df1

df.rename(columns={'in_reply_to_screen_name':'num_reply','rt_user_screen_name':'num_rt'},inplace=True,errors='ignore')

'''df['num_in_reply'] = dfReply
df.loc[df['num_in_reply'].isna(),'num_in_reply'] = 0'''

df['num_in_rt'] = dfRetweet['real_rt']
df.loc[df['num_in_rt'].isna(),'num_in_rt'] = 0

df

In [ ]:
df.describe()

In [ ]:
df.sort_values('num_in_rt',ascending=False,inplace=True)
df = df.head(10)
df

In [ ]:
n = 20
fig = make_subplots(rows=1, cols=1)

fig.add_trace(go.Bar(y=df.head(n).index, x=df.head(n)['num_reply'],orientation='h',  name = 'Reply'), row=1, col=1)
fig.add_trace(go.Bar(y=df.head(n).index, x=df.head(n)['num_in_rt'],orientation='h', name = 'Retweet'), row=1, col=1)
fig.add_trace(go.Bar(y=df.head(n).index, x=df.head(n)['sum_self_rt'],orientation='h',  name = 'Self retweet')
              , row=1, col=1)

fig.update_layout(title="How many time this users are retweeted")
fig.update_xaxes(title="Count")
fig.update_yaxes(title="Username")
fig.show()

## Comparison with noVax and high credibility users

In [ ]:
df = df_preparation([])
dfNovax = df_preparation([],filename='../tweets_novax.csv')
dfProvax = df_preparation([],filename='../tweets_provax.csv')

In [ ]:
print("There are %d (%d%%) novax that have been suspended from Twitter"%
      (len(dfNovax),(len(dfNovax)/len(df))*100))

In [ ]:
print("There are %d (%d%%) provax that have been suspended from Twitter"%
      (len(dfProvax),(len(dfProvax)/len(df))*100))